In [129]:
import pandas as pd
import numpy as np
import itertools
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn import metrics, svm, neighbors, linear_model, tree
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
%matplotlib inline

# Read data from file

In [43]:
pwd

'C:\\Users\\Katelyn\\Documents'

In [44]:
cd /Users/Katelyn/Documents/

C:\Users\Katelyn\Documents


In [45]:
df = pd.read_csv("Order_Imbalance_Data.csv")
df.head()

,Time,AskPrice,AskSize,BidPrice,BidSize,VOI,OIR,Spread,PriceChange,MPB,VOI_Norm,OIR_Norm,MPB_Norm
0,2018-07-16 07:00:00-05:00,280325,118,280300,74,0,-0.229167,25,0,0.0,0.00,-0.009167,0.0
1,2018-07-16 07:00:00.100000-05:00,280300,85,280275,35,-85,-0.416667,25,-25,-12.5,-3.40,-0.016667,-0.5
2,2018-07-16 07:00:00.200000-05:00,280300,81,280275,49,18,-0.246154,25,0,0.0,0.72,-0.009846,0.0
3,2018-07-16 07:00:00.300000-05:00,280300,81,280275,49,0,-0.246154,25,0,0.0,0.00,-0.009846,0.0
4,2018-07-16 07:00:00.400000-05:00,280300,81,280275,49,0,-0.246154,25,0,0.0,0.00,-0.009846,0.0


In [53]:
Y = ['PriceChange']
attributes = ['VOI, OIR, MPB']
#Calculate each attribute and output into .csv file
X = ['attributes'] 

In [125]:
#Y = df['PriceChange']
#features = ['VOI' , 'OIR' , 'MPB']
#features_ind = list(range(800, 70))
#X = df[features_ind]
#feature_names = X.columns.tolist()
#print(len(feature_names))
#print(feature_names)
#X.head()

#Add to this later 

# helper functions

In [106]:
# features analysis
def select_features(X, Y, plot=False):
    forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
    
    feature_names = X.columns.tolist()


    forest.fit(X, Y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    
    importances = np.array(importances)
    importances /= importances.max()

    if plot:
        for f in range(X.shape[1]):
            print("%d. feature '%s' (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))


        # Plot the feature importances of the forest
        plt.figure()
        plt.title("Feature importances")
        plt.bar(range(X.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
        plt.xticks(range(X.shape[1]), indices)
        plt.xlim([-1, X.shape[1]])
        plt.show()
        
    # select features > 25 % importance
    features = [i for i,j in zip(feature_names,importances) if j > 0.25]

    print ("Selected number of features: %d" % len(features))

    if plot:
        for f in range(len(features)):
            print("%d. feature %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))
        
    return features

In [78]:
class_names = Y.unique()
print(class_names)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

   
    print('Confusion matrix')

    

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
#Price Change attribute below in matrix 

[   0.     -25.      -1.25    -2.5     -3.75    -5.      -6.25    -7.5
   -8.75   -10.     -11.25   -12.5    -13.75   -16.25   -18.75   -21.25
  -23.75   -26.25   -28.75   -31.25   -33.75   -40.     -17.5    -20.       5.
    6.25     7.5      8.75    10.      11.25    12.5     13.75    15.
   16.25    17.5     18.75    20.      21.25    22.5     23.75    25.     -15.
  -22.5      1.25     2.5      3.75    -1.875  -11.875    1.875   16.875
   18.125   19.375   -5.625   -6.875   -8.125   -9.375  -10.625  -13.125
  -14.375  -15.625  -16.875  -18.125  -19.375    0.625    3.125    4.375
    5.625    6.875    8.125    9.375   10.625   11.875   13.125   14.375
   15.625   -0.625  -20.625  -21.875  -23.125   20.625   21.875   -3.125
   -4.375   27.5     30.      32.5     35.      37.5     40.      42.5
   45.      47.5     50.      26.25    28.75    31.25    33.75    36.25
  -24.375  -27.5    -30.     -32.5    -35.     -36.25   -38.75   -41.25
  -37.5    -42.5    -45.     -47.5    -50.      2

In [79]:
# grid search with cross-validation (subject-wise or k-folds) for picking hyperparameters
# returns list with best estimators
# params:
# X_train - training data
# y_train - labels
# kfoldcv - true to use 10-fold cross-validation
# display - boolean for printing out cross-validation info
def grid(X_train,y_train, display=False, kfoldcv=False):
    # Different models to try
    #       Model name ---------------------------------------------------------------------
    #      Parameters ------------------------------------------                           |
    #     Classifier -----------                               |                           |
    #                          |                               |                           |
    #                          v                               v                           |
    models = [[tree.DecisionTreeClassifier(), {'min_samples_split': [2, 4, 6, 8, 10],#    |
                                               'min_samples_leaf': [1, 5, 10, 15, 20],#     v
                                               'max_depth': [10, 20, 30, 40, 50]},       "Decision Tree"]
              ]

    models.append([linear_model.LogisticRegression(), {'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Ridge Penalty"])

#     models.append([linear_model.LogisticRegression(penalty='l1'), {'C': [100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Lasso Penalty"])

    # LinearSVC
    models.append([svm.SVC(kernel='rbf'), { # class_weight="balanced"; tol 
                              'gamma': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6], 
                               'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]}, "SVM rbf"])
    
    models.append([neighbors.KNeighborsClassifier(), {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}, "K-Nearest Neighbors"])


    models.append([RandomForestClassifier(), {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400]
                                               }, "Random Forest"])

    models_with_best_params = []

    # cross-validation strategy
    # Leave One Group Out
    groups = y_train.unique()
    logo = LeaveOneGroupOut()
    cv = logo.get_n_splits(X_train,y_train,groups)
    
    # for 10-folds cross-validation
    if kfoldcv:
        cv = 10 

    for model in models:
        clf = GridSearchCV(model[0], model[1], cv = cv)
        clf.fit(X_train, y_train)
        best_params = clf.best_params_
        best_estimator = clf.best_estimator_

        model_with_best_params = [best_estimator, best_params, model[2]]

        models_with_best_params.append(model_with_best_params)

        if (display):
            print(model[2], ": ")
            print("Best score for ", model[2], ":", clf.best_score_)
            print()
            print("Best parameters for ", model[2], " found on development set:", best_params)
    #         print()
    #         print("Best estimator for ", model[2], " model:", best_estimator)
            print()
    #         print()

    #         print("Grid scores on development set:")
    #         print()
    #         for params, mean_score, scores in clf.grid_scores_:
    #             print("%0.3f (+/-%0.03f) for %r"
    #                   % (mean_score, scores.std() * 2, params))
            print()

    #     print(models_with_best_params)
    
#     return best_params['n_estimators']
    return  models_with_best_params

In [80]:
def test_classifier (model, X_train, y_train, X_test, y_test):
#     classifier = RandomForestClassifier(n_estimators=pram)
    classifier = model[0]
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    #print("Prediction accuracy for", model[2], "model is", score)
    expected = y_test
    predicted = classifier.predict(X_test)

    print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted, digits=5)))
    #Compute confusion matrix
    cnf_matrix = confusion_matrix(expected, predicted)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
    title='Confusion matrix, without normalization: ' + model[2])

    plt.show()
    
    return [metrics.f1_score(expected,predicted, average='weighted'), cnf_matrix]

# 10-fold cross-validation

In [1]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=50)
print("Train Set Size: ", len(X_train))
print("Test Set Size: ", len(X_test))

features = select_features(X_train, y_train)
X_train = X_train[features]
print("Train set size after feature selection: ", X_train.shape)
X_test = X_test[features]
print("Test set size after feature selection: ", X_test.shape)

#Split 70/30

NameError: name 'train_test_split' is not defined

In [127]:
best_classifiers = grid(X_train, y_train, True, True)

NameError: name 'LeaveOneGroupOut' is not defined

In [128]:
#fitting models to test_data#fitting 
for model in best_classifiers:
    test_classifier (model, X_train, y_train, X_test, y_test)

NameError: name 'best_classifiers' is not defined

# subject-wise cross-validation

In [91]:
#Get confusion matrix
#Fix classifiers in separate file 

In [ ]:
#Random Forest 
#Linear Regression
#Logistic Regression
#SVM
#Maybe KNN and Decision trees, but not a priority rn

In [1]:
# Load digits data from 'datasets module'
digits = datasets.load_digits()
svr = svm.SVC(kernel='rbf')
parameters = {'C':c, 'gamma': g}

n_samples = np.round(len(digits.data)/2)

#   Get data-records and record-labels in arrays X and y
X_training=digits.data[:n_samples]
y_training=digits.target[:n_samples]

#   Get data-records and record-labels in arrays X and y
X_testing=digits.data[n_samples:]
y_testing=digits.target[n_samples:]

#clf = grid_search.GridSearchCV(knearest, parameters, cv=10)
clf = grid_search.GridSearchCV(svr, parameters)

#clf = svm.SVC(kernel='poly', gamma=1) 
clf.fit(X_training, y_training)

# Make class predictions for all observations in X_testing
Z = clf.predict(X_testing)

# Compare predicted class labels with actual class labels
accuracy=clf.score(X_testing,y_testing)
print ("Predicted model accuracy: "+ str(accuracy))
print (clf.best_params_['C'])
print (clf.best_params_['gamma'])
print (clf.get_params())
print (cross_validation.cross_val_score(clf, X_testing, y_testing))

NameError: name 'datasets' is not defined

In [2]:
def test_classifier (model, X_train, y_train, X_test, y_test):
#     classifier = RandomForestClassifier(n_estimators=pram)
    classifier = model[0]
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    #print("Prediction accuracy for", model[2], "model is", score)
    expected = y_test
    predicted = classifier.predict(X_test)

    print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted, digits=5)))
    #Compute confusion matrix
    cnf_matrix = confusion_matrix(expected, predicted)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
    title='Confusion matrix, without normalization: ' + model[2])

    plt.show()
    
#Split data 60/40
#Run 60% through classifiers and then test on the remaining 40% to see which produced highest accuracy results

#What is this one's accuracy?

In [3]:
# Load digits data from 'datasets module'
digits = datasets.load_digits()
c = 10.0**np.arange(-5,5)
n_samples = np.round(len(digits.data)/2)
parameters = {'C':c,}

#   Get data-records and record-labels in arrays X and y
X_training=digits.data[:n_samples]
y_training=digits.target[:n_samples]

#   Get data-records and record-labels in arrays X and y
X_testing=digits.data[n_samples:]
y_testing=digits.target[n_samples:]

svr = LogisticRegression(penalty='l1')

# Create an instance of KNeighborsClassifier and then fit training data
clf = grid_search.GridSearchCV(svr, parameters)

#clf = LogisticRegression(C=1.0, dual=False, fit_intercept=True,intercept_scaling=1, penalty='l2', tol=0.0001)
clf.fit(X_training, y_training)

# Make class predictions for all observations in X_testing
Z = clf.predict(X_testing)

# Compare predicted class labels with actual class labels
accuracy=clf.score(X_testing,y_testing)
print ("Predicted model accuracy: "+ str(accuracy))
print (clf.best_params_['C'])
print (clf.get_params())

NameError: name 'datasets' is not defined